In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
import gc
import matplotlib.pyplot as plt
from lightgbm import LGBMClassifier

### Processing Application data

In [ ]:
print(pd.read_csv("../input/application_train.csv").shape[0] + pd.read_csv("../input/application_test.csv").shape[0])

In [ ]:
application_train = pd.read_csv("../input/application_train.csv")
application_test = pd.read_csv("../input/application_test.csv")

application_train["source"] = "train"
application_test["source"] = "test"

''' 1. Append train and test data'''
application = pd.concat([application_train, application_test], axis=0, ignore_index=True)

'''2. Find columns with a lot of missing data and remove uncorrelated variables'''
missing_cols_prcnt = application_train.isnull().sum()/application_train.shape[0] * 100
high_missing_values = missing_cols_prcnt[missing_cols_prcnt > 50]
high_missing_values_index = high_missing_values.index.tolist()

correlations = application_train.corr()['TARGET'].sort_values()
corr_missing_cols = correlations.reindex(high_missing_values_index).sort_values()
missing_cols_to_be_dropped = corr_missing_cols.index.difference(corr_missing_cols[(corr_missing_cols > 0.02) | (corr_missing_cols < -0.02)].index).tolist()
len(missing_cols_to_be_dropped)

application = application.drop(missing_cols_to_be_dropped, axis=1)

'''3. Separate numerical and categorical data'''
#Make sure 'TARGET' and 'Source' are excluded
categorical_vars = [f for f in application.columns if f!='TARGET' and f!='source' and application[f].dtype=="object"]
#Check once more that all numerical columns are indeed continuous and not discrete
numerical_vars = [f for f in application.columns if f!='TARGET' and f!='source' and application[f].dtype!="object"]

'''4. DAYS_EMPLOYED discrepancy'''
application["DAYS_EMPLOYED_ANOMALY"] = 0
application["DAYS_EMPLOYED_ANOMALY"].loc[application[application["DAYS_EMPLOYED"] > 0]["DAYS_EMPLOYED"].index] = 1
application["DAYS_EMPLOYED"].loc[application[application["DAYS_EMPLOYED"] > 0]["DAYS_EMPLOYED"].index] = 0
application[["DAYS_EMPLOYED", "DAYS_EMPLOYED_ANOMALY"]].head()

'''5. XNA represents null in categiorical cols'''
xna_cols = []
for c in categorical_vars:
    if application[c][application[c]=='XNA'].shape[0] > 0:
        xna_cols.append(c)
        
for c in xna_cols:
    application[c] = application[c].replace('XNA', np.nan)
    
'''6. Amt credit missing values imputation'''
missing_goods_price_indices = application[application["AMT_GOODS_PRICE"].isnull()].index
norm_factor = np.mean(application["AMT_GOODS_PRICE"])/np.mean(application["AMT_CREDIT"])
#Replacing the missing values of AMT_GOODS_PRICE
application.loc[missing_goods_price_indices, "AMT_GOODS_PRICE"] = application.loc[missing_goods_price_indices, "AMT_CREDIT"] * norm_factor

'''7. For other numerical columns impute missing values by median(including ext_source_3)'''
numeric_cols_with_missing_vals = application[numerical_vars].isnull().sum()[application[numerical_vars].isnull().sum() > 0].index.tolist()
categorical_cols_with_missing_vals = application[categorical_vars].isnull().sum()[application[categorical_vars].isnull().sum() > 0].index.tolist()

#np.median(application["AMT_ANNUITY"].dropna())
for f in numeric_cols_with_missing_vals:
    application[f] = application[f].replace(np.nan, np.median(application[f].dropna()))
    
# application["CODE_GENDER"].value_counts().idxmax()
for f in categorical_cols_with_missing_vals:
    max_count = application[f].value_counts().idxmax()
    application[f] = application[f].fillna(max_count)
    
'''8. Adding domain specific columns as mentioned in other kernels'''
application['CREDIT_INCOME_PERCENT'] = application['AMT_CREDIT'] / application['AMT_INCOME_TOTAL']
application['ANNUITY_INCOME_PERCENT'] = application['AMT_ANNUITY'] / application['AMT_INCOME_TOTAL']
application['CREDIT_TERM'] = application['AMT_ANNUITY'] / application['AMT_CREDIT']
application['DAYS_EMPLOYED_PERCENT'] = application['DAYS_EMPLOYED'] / application['DAYS_BIRTH']

'''9. One hot encoding of categorical variables'''
dummies = pd.get_dummies(application[categorical_vars])
application = pd.concat([application, dummies], axis=1)
application = application.drop(categorical_vars, axis=1)
application.head()

In [ ]:
application.shape[0]

### Processing Bureau and Bureau_Balance datasets

In [ ]:
'''1. Bureau_balance'''
bureau_balance = pd.read_csv("../input/bureau_balance.csv")

#Get dummy variables for STATUS
dummies = pd.get_dummies(bureau_balance['STATUS'])
dummies.columns = ['STATUS_'+val for val in dummies.columns.values]
bureau_balance = pd.concat([bureau_balance, dummies], axis=1)
bureau_balance = bureau_balance.drop('STATUS', axis=1)

#Aggregate the dataframe at SK_ID_BUREAU level
groupby_dict = {'MONTHS_BALANCE':[np.size]}
for col in dummies.columns:
    groupby_dict[col] = sum
bureau_balance_agg = bureau_balance.groupby('SK_ID_BUREAU').agg(groupby_dict)
bureau_balance_agg.columns = [col+"_"+func for col, func in bureau_balance_agg.columns.values]
bureau_balance_agg = bureau_balance_agg.reset_index()

#Calling garbage collector
gc.enable()
del bureau_balance, dummies
gc.disable()

# bureau_balance_agg.head()
'''Bureau dataset'''
bureau = pd.read_csv("../input/bureau.csv")
bureau.head()
bureau = bureau.merge(right=bureau_balance_agg, right_on='SK_ID_BUREAU', left_on='SK_ID_BUREAU')

#Replacing NA's in DAYS_CREDIT_ENDDATE
active_cred = float(bureau[bureau["CREDIT_ACTIVE"] == 'Active'][["DAYS_CREDIT_ENDDATE"]].median())
closed_cred = float(bureau[bureau["CREDIT_ACTIVE"] == 'Closed'][["DAYS_CREDIT_ENDDATE"]].median())
bureau.loc[bureau[bureau["CREDIT_ACTIVE"] == 'Active'][bureau["DAYS_CREDIT_ENDDATE"].isnull()].index, "DAYS_CREDIT_ENDDATE"] = active_cred
bureau.loc[bureau[bureau["CREDIT_ACTIVE"] == 'Closed'][bureau["DAYS_CREDIT_ENDDATE"].isnull()].index, "DAYS_CREDIT_ENDDATE"] = closed_cred

#For days_enddate_fact, we will be taking average of only the closed credits
#So will be aggregating only at closed level and then merging to the aggregated dataset at SK_ID_CURR
#Missing value imputation will be done only for closed credits
enddate_closed_median = float(bureau[bureau["CREDIT_ACTIVE"] == 'Closed'][["DAYS_ENDDATE_FACT"]].median())
bureau.loc[bureau[bureau["CREDIT_ACTIVE"] == 'Closed'][bureau["DAYS_ENDDATE_FACT"].isnull()].index, "DAYS_ENDDATE_FACT"] = enddate_closed_median

#Creating dummy variables
bureau_dummies = pd.get_dummies(bureau[["CREDIT_ACTIVE", "CREDIT_CURRENCY", "CREDIT_TYPE"]])
bureau = pd.concat([bureau, bureau_dummies], axis=1)
bureau = bureau.drop(["CREDIT_ACTIVE", "CREDIT_CURRENCY", "CREDIT_TYPE"], axis=1)

#Aggregating Bureau data
bureau_cols = bureau.columns.tolist()
numeric_cols_mean = ['DAYS_CREDIT', 'CREDIT_DAY_OVERDUE',
       'DAYS_CREDIT_ENDDATE', 'DAYS_ENDDATE_FACT', 'AMT_CREDIT_MAX_OVERDUE', 'AMT_CREDIT_SUM_OVERDUE', 'DAYS_CREDIT_UPDATE',
       'AMT_ANNUITY',]
credit_sum_cols = ['AMT_CREDIT_SUM', 'AMT_CREDIT_SUM_DEBT',
       'AMT_CREDIT_SUM_LIMIT',]
status_cols = [col for col in bureau_cols if col.find('STATUS')!=-1]
credit_cols = [col for col in bureau_cols if col.find('CREDIT_TYPE')!=-1 or col.find('CREDIT_ACTIVE')!=-1 or col.find('CREDIT_CURRENCY')!=-1]
agg_dict = {}
for col in status_cols:
    agg_dict[col] = np.mean
for col in credit_cols:
    agg_dict[col] = sum 
for col in numeric_cols_mean:
    agg_dict[col] = np.mean
for col in credit_sum_cols:
    agg_dict[col] = [np.mean, sum]
bureau_agg = bureau.groupby('SK_ID_CURR').agg(agg_dict)
bureau_agg.columns = [col+"_"+func for col, func in bureau_agg.columns.values]
bureau_agg = bureau_agg.reset_index()

gc.enable()
del bureau
gc.disable()

bureau_agg.head()

In [ ]:
application = application.merge(bureau_agg, how='left', left_on="SK_ID_CURR", right_on="SK_ID_CURR")
gc.enable()
del bureau_agg
gc.disable()
application.head()

### Processing Previous Applications

In [ ]:
prev_application = pd.read_csv("../input/previous_application.csv")
# Days 365.243 values -> nan
prev_application['DAYS_FIRST_DRAWING'].replace(365243, np.nan, inplace= True)
prev_application['DAYS_FIRST_DUE'].replace(365243, np.nan, inplace= True)
prev_application['DAYS_LAST_DUE_1ST_VERSION'].replace(365243, np.nan, inplace= True)
prev_application['DAYS_LAST_DUE'].replace(365243, np.nan, inplace= True)
prev_application['DAYS_TERMINATION'].replace(365243, np.nan, inplace= True)

#Missing value treatment
missing_values_prcnt = prev_application.isnull().sum()/prev_application.shape[0] * 100
high_prop_missing_cols = list(missing_values_prcnt[missing_values_prcnt > 50].index)
#Dropping columns with high missing values proportion
prev_application = prev_application.drop(high_prop_missing_cols, axis=1) 
missing_values_cols_sample = list(prev_application.isnull().sum()[prev_application.isnull().sum() > 0].index)

#Missing value treatment; Numerical features will be treated by inserting 0's and Categorical features will be treated by inputting the mode
#Actually a better technique for categorical features will be mode based on grouping SK_ID_CURR
#Missing value imputation
for col in missing_values_cols_sample:
    if prev_application[col].dtype != "object":
        prev_application[col] = prev_application[col].fillna(0)
    elif prev_application[col].dtype == "object":
        prev_application[col] = prev_application[col].mode()


#Hour process column is actually categorical. Let's convert it into one
prev_application['HOUR_APPR_PROCESS_START'] = prev_application['HOUR_APPR_PROCESS_START'].map(str)
categorical_features = [f for f in prev_application.columns if prev_application[f].dtype=="object"]
prev_application_with_dummy = pd.get_dummies(prev_application, columns=categorical_features)
dummy_columns = [f for f in prev_application_with_dummy.columns if f not in prev_application.columns]#Getting the list of all newly created dummy columns


#Rolling up values at sk_id_curr level. All the columns will use mean as aggregation function 
prev_application_with_dummy = prev_application_with_dummy.drop('SK_ID_PREV', axis=1)
prev_agg = prev_application_with_dummy.groupby('SK_ID_CURR').agg('mean')
prev_agg = prev_agg.reset_index()

gc.enable()
del prev_application, prev_application_with_dummy
gc.disable()

prev_agg.head()

In [ ]:
application = application.merge(prev_agg, how='left', left_on='SK_ID_CURR', right_on='SK_ID_CURR')
gc.enable()
del prev_agg
gc.disable()
application.head()

## Processing POS CASH BALANCE

In [ ]:
#Aggregating at SK_ID_PREV level
pos_cash = pd.read_csv("../input/POS_CASH_balance.csv")
sk_id_df = pos_cash[['SK_ID_PREV', 'SK_ID_CURR']].drop_duplicates()

pos_cash = pos_cash.drop(['MONTHS_BALANCE', 'CNT_INSTALMENT_FUTURE'], axis=1)

pos_cash = pd.get_dummies(pos_cash)
pos_cash = pos_cash.drop(['NAME_CONTRACT_STATUS_Active', 'NAME_CONTRACT_STATUS_Signed'], axis=1)

agg_dict = {
    'CNT_INSTALMENT': np.mean, 
    'SK_DPD': [sum, np.mean], 
    'SK_DPD_DEF': [sum, np.mean], 
}
name_contract_cols = [col for col in pos_cash.columns if col.find('NAME_CONTRACT_STATUS_')!= -1]
for col in name_contract_cols:
    agg_dict[col] = sum
pos_cash_agg = pos_cash.groupby('SK_ID_PREV').agg(agg_dict)

gc.enable()
del pos_cash
gc.disable()

pos_cash_agg.columns = [col+"_"+func for col, func in pos_cash_agg.columns] 
pos_cash_agg = pos_cash_agg.reset_index()
pos_cash_final = pos_cash_agg.merge(sk_id_df, how='left', left_on='SK_ID_PREV', right_on='SK_ID_PREV')

gc.enable()
del pos_cash_agg, sk_id_df
gc.disable()

pos_cash_final.head()

In [ ]:
#Aggregating at SK_ID_CURR level
mean_cols = [col for col in pos_cash_final.columns if col.find('mean')!= -1]
sum_cols = [col for col in pos_cash_final.columns if col.find('sum')!= -1]
agg_dict = {}
for col in mean_cols:
    agg_dict[col] = np.mean
for col in sum_cols:
    agg_dict[col] = sum
pos_cash_curr_lvl = pos_cash_final.groupby('SK_ID_CURR').agg(agg_dict)
pos_cash_curr_lvl = pos_cash_curr_lvl.reset_index()
gc.enable()
del pos_cash_final
gc.disable()
pos_cash_curr_lvl.head()

In [ ]:
application = application.merge(pos_cash_curr_lvl, how='left', left_on='SK_ID_CURR', right_on='SK_ID_CURR')
gc.enable()
del pos_cash_curr_lvl
gc.disable()
application.head()

## Processing Installments Data

In [ ]:
#Aggregating at SK_ID_PREV level
installments = pd.read_csv("../input/installments_payments.csv")
#if days_delay is a positive number then that's a bad sign
installments['DAYS_DELAY'] = installments['DAYS_ENTRY_PAYMENT'] - installments['DAYS_INSTALMENT']
#if positive value then it's bad
installments['AMT_DEBT'] = installments['AMT_INSTALMENT'] - installments['AMT_PAYMENT']
agg_dict = {
    'NUM_INSTALMENT_VERSION': [np.size], 
    'DAYS_DELAY': [sum, np.mean, max], 
    'AMT_DEBT': [ max, sum, np.mean]
}
installments_agg = installments.groupby('SK_ID_PREV').agg(agg_dict)
installments_agg.columns = [col+'_'+func for col, func in installments_agg.columns]
installments_agg = installments_agg.reset_index()
sk_id_df = installments[['SK_ID_PREV', 'SK_ID_CURR']].drop_duplicates()
installments_agg = installments_agg.merge(sk_id_df, how='left', left_on='SK_ID_PREV', right_on='SK_ID_PREV')

#Aggregating at SK_ID_CURR level
mean_cols = [col for col in installments_agg.columns if col.find('mean')!= -1]
sum_cols = [col for col in installments_agg.columns if col.find('sum')!= -1]
max_cols = [col for col in installments_agg.columns if col.find('max')!= -1]
agg_dict = {}
for col in mean_cols:
    agg_dict[col] = np.mean
for col in sum_cols:
    agg_dict[col] = sum
for col in max_cols:
    agg_dict[col] = max
agg_dict['NUM_INSTALMENT_VERSION_size'] = np.mean
installments_sk_curr_lvl = installments_agg.groupby('SK_ID_CURR').agg(agg_dict).reset_index()
gc.enable()
del installments, installments_agg, sk_id_df
gc.disable()
installments_sk_curr_lvl.head()

In [ ]:
application = application.merge(installments_sk_curr_lvl, how='left', left_on='SK_ID_CURR', right_on='SK_ID_CURR')
gc.enable()
del installments_sk_curr_lvl
gc.disable()
application.head()

### Processing Credit Card Balance dataset

In [ ]:
cc = pd.read_csv('../input/credit_card_balance.csv')
cc = pd.get_dummies(cc)
# General aggregations
cc.drop(['SK_ID_PREV'], axis= 1, inplace = True)
cc_agg = cc.groupby('SK_ID_CURR').agg(['min', 'max', 'mean', 'sum', 'var'])
cc_agg.columns = pd.Index(['CC_' + e[0] + "_" + e[1].upper() for e in cc_agg.columns.tolist()])
# Count credit card lines
cc_agg['CC_COUNT'] = cc.groupby('SK_ID_CURR').size()
cc_agg = cc_agg.reset_index()
del cc
gc.collect()
cc_agg.head()

In [ ]:
application = application.merge(cc_agg, how='left', left_on='SK_ID_CURR', right_on='SK_ID_CURR')
gc.enable()
del cc_agg
gc.disable()
application.head()

16) LGBM K FOLD

In [ ]:
X_train = application[application["source"] == 'train'].drop(["source", "TARGET"], axis=1)
Y_train = application[application["source"] == 'train']["TARGET"]
X_test = application[application["source"] == 'test'].drop(["source", "TARGET"], axis=1)

In [ ]:
X_test.shape

In [ ]:
k_fold = KFold(n_splits = 5, shuffle = True, random_state = 50)
x_train = np.array(X_train.values)
y_train = np.array(Y_train.values)
x_test = np.array(X_test.values)
test_predictions = np.zeros(x_test.shape[0])
train_auc = []
valid_auc = []
for train_indices, valid_indices in k_fold.split(x_train):
    train_data, train_target = x_train[train_indices], y_train[train_indices]
    valid_data, valid_target = x_train[valid_indices], y_train[valid_indices]
    clf = LGBMClassifier(
        n_estimators=1000,
        learning_rate=0.03,
        num_leaves=30,
        colsample_bytree=.8,
        subsample=.9,
        max_depth=7,
        reg_alpha=.1,
        reg_lambda=.1,
        min_split_gain=.01,
        min_child_weight=2,
        silent=-1,
        verbose=-1,
    )
        
    clf.fit(train_data, train_target, 
        eval_set= [(train_data, train_target), (valid_data, valid_target)], 
        eval_names = ['train', 'valid'],
        eval_metric='auc', verbose=100, early_stopping_rounds=100  #30
   )
    
    #best_iteration = clf.best_iteration_
    
    valid_score = clf.best_score_['valid']['auc']
    train_score = clf.best_score_['train']['auc']
    valid_auc.append(valid_score)
    train_auc.append(train_score)
    
    test_predictions += clf.predict_proba(x_test, num_iteration=clf.best_iteration_)[:,1]/k_fold.n_splits
    
    gc.enable()
    del clf, train_data, valid_data
    gc.disable()
        

In [ ]:
kfold_auc = pd.DataFrame({'train_auc': train_auc, 
                          'valid_auc': valid_auc})
kfold_auc

In [ ]:
test_output = pd.DataFrame({'SK_ID_CURR':X_test.SK_ID_CURR.values, 'TARGET':test_predictions})
test_output.head()

In [ ]:
test_output.shape

In [ ]:
test_output.to_csv('light_gbm_v7.csv', index = False)